# Notebook

El siguiente set de datos fue obtenido a través de un sitio web que creé. En este sitio los participantes podían realizar un Test OCEAN y luego visualizar los resultados que obtenían. Además de las 50 preguntas estándar del test debían contestar distintas preguntas con fines demográficos. Estas eran:
- ¿Qué tan feliz te sentís hoy?
- ¿Qué carrera universitaria cursaste o estás cursando?
- ¿Con qué género te identificas?
- Fecha de nacimiento
- País de nacimiento
- Provincia de nacimiento
- Ciudad de nacimiento

El siguiente notebook busca crear a partir de estos datos nuevas columnas que luego serán utilizadas con el fin de realizar un análisis estadístico de los datos.

Dado que para el analísis se creyó importante contar con datos de personas de distintas partes del mundo, el sitio web estaba en Español, Inglés, Italiano, Francés y Catalán. Esto generó que las columnas "degree_category" y "birth_location_country" tengan data de tipo texto muy variada. Para poder agrupar las carreras y países en grupos las columnas "degree_category" y "birth_location_country_clean" fueron creadas y llenadas manualmente.

Para "degree_category" primero se hizo una selección de 8 grupos de estudios donde pudiera caer cada fila. Estos fueron:
- STEM (Por sus siglas en inglés de Science, Technology, Engineering and Mathematics). Por ejemplo Ingeniería, Física, Arquitectura.
- Negocios. Por ejemplo Contador Público, Administración, Marketing.
- Ciencias Sociales. Por ejemplo Abogacía, Economía, Ciencia Política.
- Artes. Por ejemplo Bellas Artes, Música, Diseño Gráfico.
- Ciencias de la salud. Por ejemplo Medicina, Veterinaria, Educación Física.
- Educación. Por ejemplo Docentes, Profesorados, Licenciados en Educación.
- Humanidades. Por ejemplo Historia, Filosofía, Traductorados.
- Otros. Por ejemplo Gastronomía, Decoración de interiores, N/A.

Luego y con la ayuda de una traducción, manualmente se colocó a cada uno la categoría correspondiente.

En el caso de la columna "birth_location_country_clean" se creó para poder agrupar respuestas que significaban lo mismo pero estaban escritas de distinta manera. Por ejemplo "España/Spain/Espanya". 

In [1]:
import pandas as pd

In [2]:
csv_file_path = 'data/data_semi_raw.csv'
data_raw = pd.read_csv(csv_file_path)

data = data_raw.copy()

print('Cantidad de participantes: ', len(data))
data.head()

Cantidad de participantes:  604


,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,opn10,degree_category,degree,happiness_level,gender,birth_date,birth_location_country,birth_location_country_clean,birth_location_province,birth_location_city
0,4,1,4,2,4,2,4,2,4,2,...,5,Business,Negocios Digitales,4,M,1998/08/15,Argentina,Argentina,Buenos Aires,San Isidro
1,4,1,3,2,5,1,3,3,2,4,...,5,STEM,Física,4,M,1997/04/14,Argentina,Argentina,CABA,Recoleta
2,4,3,4,2,4,2,4,2,2,4,...,3,Social Sciences,Dret,3,F,2000/01/28,Catalunya,Spain,Barcelona,Barcelona
3,2,3,4,3,3,4,4,3,3,3,...,4,Education,Psicopedagogia,3,F,1967/01/04,Argentina,Argentina,Ciudad de Buenos Aires,Ciudad de Buenos Aires
4,3,4,2,3,5,1,5,4,1,5,...,5,NaN,Ninguna,3,M,1995/02/24,Argentina,Argentina,Buenos aires,San isidro


Primero vamos a eliminar todas las columnas que no vamos a usar en nuestro análisis después.
- degree
- birth_location_country
- birth_location_province
- birth_location_city 

In [3]:
data.drop(columns=['degree', 'birth_location_country', 'birth_location_province', 'birth_location_city'], inplace=True)

data.sample(5)

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,opn6,opn7,opn8,opn9,opn10,degree_category,happiness_level,gender,birth_date,birth_location_country_clean
403,1,3,4,4,5,4,5,3,4,4,...,5,2,1,1,3,Health Sciences,4,M,2023/10/26,Argentina
219,3,2,4,2,4,2,2,2,4,3,...,3,4,4,5,4,Education,4,F,1976/12/06,Argentina
36,4,1,3,2,5,2,4,2,4,2,...,1,4,5,4,5,Arts,3,F,2000/08/06,Argentina
315,3,3,4,4,4,1,4,5,3,4,...,3,4,2,1,2,STEM,4,M,1966/03/31,Argentina
183,1,3,3,4,4,1,3,5,1,4,...,3,4,1,2,3,STEM,4,M,1999/08/31,Argentina


Dentro de las 50 preguntas que se realizaban, 10 corresponden a cada dimensión de la personalidad. El score de cada uno de estos rasgos se calcula sumando el total de las respuestas de las preguntas que componen al rasgo. Como algunas preguntas son negadas nos estarían sumando o restando más puntos de los que deberían.

Por ejemplo:

'ext1' : 'Soy el alma de la fiesta.',
'ext2' : 'No hablo mucho.',
'ext3' : 'Me siento cómodo cuando estoy rodeado de gente.',
'ext4' : 'Me mantengo en segundo plano.',
'ext5' : 'Empiezo conversaciones.',
'ext6' : 'Tengo poco para decir.',
'ext7' : 'Hablo con mucha gente distinta en una fiesta.',
'ext8' : 'No me gusta llamar la atención.',
'ext9' : 'No me molesta ser el centro de atención.',
'ext10': 'Soy tímido cuando estoy con extraños.'

Las preguntas 'ext2', 'ext4', 'ext6', 'ext8', 'ext10' son preguntadas negadas. Una respuesta baja en estas preguntas en realidad debería ser un score alto.

Para solucionar esto vamos a hacer un re-encoding de las columnas que son preguntadas negadas. El puntaje que cada persona le otorgó va a ser reemplazado por 6 - dicho puntaje. Por ejemplo, si el puntaje que la persona dio fue 1, será reemplazado por 6 - 1, lo cual da 5. Si fue 5, será reemplazado por 6 - 5, lo cual da 1.

La lista completa de preguntas es la siguiente:

1. Soy el alma de la fiesta.
2. No hablo mucho.
3. Me siento cómodo cuando estoy rodeado de gente.
4. Me mantengo en segundo plano.
5. Empiezo conversaciones.
6. Tengo poco para decir.
7. Hablo con mucha gente distinta en una fiesta.
8. No me gusta llamar la atención.
9. No me molesta ser el centro de atención.
10. Soy tímido cuando estoy con extraños.
11. Me estreso fácilmente.
12. Soy relajado la gran parte del tiempo.
13. Me preocupo por cosas.
14. Rara vez me siento triste.
15. Soy fácilmente perturbado.
16. Me decepciono fácilmente.
17. Mi estado de ánimo cambia mucho.
18. Tengo frecuentes cambios de ánimo.
19. Me irrito fácilmente.
20. Usualmente me siento triste.
21. Me preocupo muy poco por los demás.
22. Me interesa la gente.
23. Insulto a la gente.
24. Simpatizo con los sentimientos de los demás.
25. No me interesan los problemas de otras personas.
26. Tengo un corazón suave.
27. No me interesan los demás.
28. Me hago el tiempo para los demás.
29. Siento las emociones de los demás.
30. Hago que la gente se sienta a gusto.
31. Siempre estoy preparado.
32. Dejo mis pertenencias dispersas.
33. Presto atención a los detalles.
34. Hago un lío de las cosas.
35. Hago mis tareas lo más pronto posible.
36. Usualmente olvido poner las cosas de vuelta en su lugar.
37. Me gusta el orden.
38. Eludo mis deberes.
39. Sigo un horario.
40. Soy exigente en mi trabajo.
41. Tengo un vocabulario rico en palabras.
42. Me resulta difícil entender ideas abstractas.
43. Tengo una imaginación muy fuerte.
44. No me interesan las ideas abstractas.
45. Tengo ideas excelentes.
46. No tengo una buena imaginación.
47. Soy rápido para entender las cosas.
48. Uso palabras complicadas.
49. Me paso mucho tiempo reflexionado sobre cosas.
50. Estoy lleno de ideas.

In [4]:
columns_to_reencode = ['ext2', 'ext4', 'ext6', 'ext8', 'ext10',
                       'est2', 'est4',
                       'agr1', 'agr3', 'agr5', 'agr7',
                       'csn2', 'csn4', 'csn6', 'csn8', 
                       'opn2', 'opn4', 'opn6']

data[columns_to_reencode] = 6 - data[columns_to_reencode]

# parseamos a float las respuestas para después poder calcular distintas cosas
data.iloc[:, 0:50] = data.iloc[:, 0:50].astype(float)

data.head()

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,opn6,opn7,opn8,opn9,opn10,degree_category,happiness_level,gender,birth_date,birth_location_country_clean
0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,5.0,4.0,5.0,5.0,5.0,Business,4,M,1998/08/15,Argentina
1,4.0,5.0,3.0,4.0,5.0,5.0,3.0,3.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,STEM,4,M,1997/04/14,Argentina
2,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,...,3.0,5.0,5.0,4.0,3.0,Social Sciences,3,F,2000/01/28,Spain
3,2.0,3.0,4.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,...,3.0,4.0,3.0,4.0,4.0,Education,3,F,1967/01/04,Argentina
4,3.0,2.0,2.0,3.0,5.0,5.0,5.0,2.0,1.0,1.0,...,4.0,4.0,2.0,5.0,5.0,NaN,3,M,1995/02/24,Argentina


1. Creando columna 'age'

In [5]:
from datetime import datetime

current_year = datetime.now().year
data['birth_date'] = pd.to_datetime(data['birth_date'], format='%Y/%m/%d')
data['age'] = current_year - data['birth_date'].dt.year

data.sample(5)

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,opn7,opn8,opn9,opn10,degree_category,happiness_level,gender,birth_date,birth_location_country_clean,age
80,4.0,4.0,3.0,3.0,4.0,5.0,3.0,3.0,3.0,5.0,...,4.0,4.0,4.0,4.0,Business,3,M,1997-05-20,Chile,26
260,2.0,2.0,2.0,2.0,1.0,2.0,1.0,3.0,2.0,1.0,...,4.0,1.0,2.0,3.0,Business,3,M,1997-10-31,Argentina,26
248,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,5.0,3.0,...,4.0,4.0,4.0,4.0,STEM,4,F,1979-09-22,Argentina,44
342,4.0,5.0,5.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,5.0,5.0,5.0,5.0,STEM,4,M,1999-04-18,Argentina,24
281,4.0,5.0,3.0,3.0,3.0,4.0,1.0,1.0,1.0,2.0,...,5.0,4.0,5.0,5.0,STEM,5,F,1992-07-14,Chile,31


2. Creando columna de age-groups

In [6]:
bins = [0, 18, 30, 45, 60, 105]
labels = ['0-18','18-30', '31-45', '46-59', '60+']
data['age_group'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

data.sample(5)

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,opn8,opn9,opn10,degree_category,happiness_level,gender,birth_date,birth_location_country_clean,age,age_group
239,4.0,4.0,5.0,4.0,5.0,4.0,5.0,5.0,4.0,4.0,...,5.0,4.0,3.0,Social Sciences,4,F,2000-06-02,Croatia,23,18-30
302,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,1.0,5.0,...,2.0,5.0,5.0,STEM,5,M,1996-07-17,Argentina,27,18-30
13,3.0,3.0,3.0,4.0,5.0,4.0,4.0,3.0,3.0,2.0,...,5.0,5.0,2.0,Social Sciences,4,F,1979-12-31,Argentina,44,31-45
47,4.0,2.0,4.0,2.0,4.0,2.0,3.0,4.0,3.0,3.0,...,2.0,4.0,4.0,Other,3,F,1986-11-09,Italy,37,31-45
73,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,NaN,1,M,2023-05-02,Italy,0,0-18


3. Creando columnas de score por rasgo de personalidad

In [9]:
openness_columns = ['opn1', 'opn2', 'opn3', 'opn4', 'opn5', 'opn6', 'opn7', 'opn8', 'opn9', 'opn10']
conscientiousness_columns = ['csn1', 'csn2', 'csn3', 'csn4', 'csn5', 'csn6', 'csn7', 'csn8', 'csn9', 'csn10']
extraversion_columns = ['ext1', 'ext2', 'ext3', 'ext4', 'ext5', 'ext6', 'ext7', 'ext8', 'ext9', 'ext10']
agreeableness_columns = ['agr1', 'agr2', 'agr3', 'agr4', 'agr5', 'agr6', 'agr7', 'agr8', 'agr9', 'agr10']
neuroticism_columns = ['est1', 'est2', 'est3', 'est4', 'est5', 'est6', 'est7', 'est8', 'est9', 'est10']

data['openness_score'] = data[openness_columns].sum(axis=1)
data['conscientiousness_score'] = data[conscientiousness_columns].sum(axis=1)
data['extraversion_score'] = data[extraversion_columns].sum(axis=1)
data['agreeableness_score'] = data[agreeableness_columns].sum(axis=1)
data['neuroticism_score'] = data[neuroticism_columns].sum(axis=1)

data.sample(5)

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,gender,birth_date,birth_location_country_clean,age,age_group,openness_score,conscientiousness_score,extraversion_score,agreeableness_score,neuroticism_score
547,2.0,2.0,3.0,2.0,3.0,2.0,2.0,2.0,4.0,2.0,...,F,2004-05-26,Argentina,19,18-30,28.0,43.0,24.0,43.0,22.0
363,3.0,2.0,2.0,2.0,3.0,4.0,2.0,1.0,4.0,2.0,...,M,1992-07-05,Argentina,31,31-45,49.0,36.0,25.0,39.0,30.0
224,4.0,2.0,1.0,2.0,5.0,3.0,5.0,1.0,1.0,4.0,...,F,1978-07-08,Argentina,45,46-59,33.0,39.0,28.0,33.0,39.0
258,1.0,5.0,5.0,3.0,4.0,5.0,5.0,4.0,1.0,3.0,...,F,1958-03-07,Argentina,65,60+,46.0,40.0,36.0,48.0,14.0
91,3.0,2.0,3.0,2.0,4.0,5.0,3.0,2.0,2.0,3.0,...,M,1999-01-19,Argentina,24,18-30,40.0,38.0,29.0,43.0,30.0


In [8]:
# what are the quartiles for age groups??

data['age'].quantile([0.25, 0.5, 0.75])

0.25    24.0
0.50    29.0
0.75    43.0
Name: age, dtype: float64

4. Creando columna de signo zodiacal

In [12]:
# Función para obtener el signo zodiacal según la fecha de nacimiento
def get_zodiac_sign(birth_date):
    if not isinstance(birth_date, str):
        birth_date = birth_date.strftime("%Y/%m/%d")
        
    birth_date = datetime.strptime(birth_date, "%Y/%m/%d")
    day = birth_date.day
    month = birth_date.month

    if ((month == 1 and day >= 20) or (month == 2 and day <= 18)):
        return "Acuario"
    elif ((month == 2 and day >= 19) or (month == 3 and day <= 20)):
        return "Piscis"
    elif ((month == 3 and day >= 21) or (month == 4 and day <= 19)):
        return "Aries"
    elif ((month == 4 and day >= 20) or (month == 5 and day <= 20)):
        return "Tauro"
    elif ((month == 5 and day >= 21) or (month == 6 and day <= 20)):
        return "Géminis"
    elif ((month == 6 and day >= 21) or (month == 7 and day <= 22)):
        return "Cáncer"
    elif ((month == 7 and day >= 23) or (month == 8 and day <= 22)):
        return "Leo"
    elif ((month == 8 and day >= 23) or (month == 9 and day <= 22)):
        return "Virgo"
    elif ((month == 9 and day >= 23) or (month == 10 and day <= 22)):
        return "Libra"
    elif ((month == 10 and day >= 23) or (month == 11 and day <= 21)):
        return "Escorpio"
    elif ((month == 11 and day >= 22) or (month == 12 and day <= 21)):
        return "Sagitario"
    elif ((month == 12 and day >= 22) or (month == 1 and day <= 19)):
        return "Capricornio"

# Aplicamos la función a cada participante en la nueva columna 'zodiac_sign'
data['zodiac_sign'] = data['birth_date'].apply(get_zodiac_sign)

data.head()

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,birth_location_country_clean,age,age_group,openness_score,conscientiousness_score,extraversion_score,agreeableness_score,neuroticism_score,zodiac_sign,zodiac_group
0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,Argentina,25,18-30,48.0,33.0,41.0,42.0,28.0,Leo,Fuego
1,4.0,5.0,3.0,4.0,5.0,5.0,3.0,3.0,2.0,2.0,...,Argentina,26,18-30,50.0,35.0,36.0,44.0,35.0,Aries,Fuego
2,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,...,Spain,23,18-30,42.0,31.0,35.0,31.0,40.0,Acuario,Aire
3,2.0,3.0,4.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,...,Argentina,56,46-59,36.0,31.0,30.0,42.0,36.0,Capricornio,Tierra
4,3.0,2.0,2.0,3.0,5.0,5.0,5.0,2.0,1.0,1.0,...,Argentina,28,18-30,43.0,33.0,29.0,41.0,20.0,Piscis,Agua


5. Creando columna de grupo zodiacal

In [13]:
# Función para obtener el grupo zodiacal según el signo zodiacal
def get_zodiac_group(zodiac_sign):
    fire_signs = ['Aries', 'Leo', 'Sagitario']
    earth_signs = ['Tauro', 'Virgo', 'Capricornio']
    air_signs = ['Géminis', 'Libra', 'Acuario']
    water_signs = ['Cáncer', 'Escorpio', 'Piscis']

    if zodiac_sign in fire_signs:
        return 'Fuego'
    elif zodiac_sign in earth_signs:
        return 'Tierra'
    elif zodiac_sign in air_signs:
        return 'Aire'
    elif zodiac_sign in water_signs:
        return 'Agua'

# Aplicamos la función a cada participante en la nueva columna 'zodiac_group'
data['zodiac_group'] = data['zodiac_sign'].apply(get_zodiac_group)

data.head()

,ext1,ext2,ext3,ext4,ext5,ext6,ext7,ext8,ext9,ext10,...,birth_location_country_clean,age,age_group,openness_score,conscientiousness_score,extraversion_score,agreeableness_score,neuroticism_score,zodiac_sign,zodiac_group
0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,Argentina,25,18-30,48.0,33.0,41.0,42.0,28.0,Leo,Fuego
1,4.0,5.0,3.0,4.0,5.0,5.0,3.0,3.0,2.0,2.0,...,Argentina,26,18-30,50.0,35.0,36.0,44.0,35.0,Aries,Fuego
2,4.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,2.0,...,Spain,23,18-30,42.0,31.0,35.0,31.0,40.0,Acuario,Aire
3,2.0,3.0,4.0,3.0,3.0,2.0,4.0,3.0,3.0,3.0,...,Argentina,56,46-59,36.0,31.0,30.0,42.0,36.0,Capricornio,Tierra
4,3.0,2.0,2.0,3.0,5.0,5.0,5.0,2.0,1.0,1.0,...,Argentina,28,18-30,43.0,33.0,29.0,41.0,20.0,Piscis,Agua


Por último, guardamos el CSV.

In [ ]:
# save data as csv file

data.to_csv('data/data_all_clean.csv', index=False)